In [8]:
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree

def get_boundary_vertex_indices(mesh: o3d.geometry.TriangleMesh):
    triangles = np.asarray(mesh.triangles)
    edge_dict = {}
    for tri in triangles:
        edges = [(tri[0], tri[1]), (tri[1], tri[2]), (tri[2], tri[0])]
        for e in edges:
            e_sorted = tuple(sorted(e))
            if e_sorted in edge_dict:
                edge_dict[e_sorted] += 1
            else:
                edge_dict[e_sorted] = 1
    boundary_edges = [e for e, count in edge_dict.items() if count == 1]
    boundary_indices = np.unique(np.array(boundary_edges).flatten())
    return boundary_indices

# 1. 메쉬 불러오기
mesh = o3d.io.read_triangle_mesh("wing.obj")
mesh.compute_vertex_normals()
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()

# 2. 내부 균일 샘플링
pcd_interior = mesh.sample_points_uniformly(number_of_points=500)

# 3. 전체 고밀도 샘플링
pcd_dense = mesh.sample_points_poisson_disk(number_of_points=5000)

# 4. 경계 vertex 추출
boundary_indices = get_boundary_vertex_indices(mesh)
boundary_vertices = np.asarray(mesh.vertices)[boundary_indices]

# 5. 거리 기반 필터링
dense_points = np.asarray(pcd_dense.points)
tree = cKDTree(boundary_vertices)
distances, _ = tree.query(dense_points)

threshold = 0.002  # 거리 임계값 (단위는 모델 스케일에 맞게 조정)
edge_points = dense_points[distances < threshold]

pcd_edge = o3d.geometry.PointCloud()
pcd_edge.points = o3d.utility.Vector3dVector(edge_points)

# 6. 병합 및 저장
pcd_all = pcd_interior + pcd_edge
o3d.io.write_point_cloud("wing_edge_dense.xyz", pcd_all, write_ascii=True)
o3d.visualization.draw_geometries([pcd_all])


In [16]:
import open3d as o3d
import numpy as np
from scipy.spatial import ConvexHull

# 1. 원본 점군 로딩
pcd = o3d.io.read_point_cloud("wing.xyz")
points = np.asarray(pcd.points)

# 2. Convex Hull으로 가장자리 판별
hull = ConvexHull(points)
edge_indices = np.unique(hull.vertices)
edge_points = points[edge_indices]

# 3. jitter 보간 함수
def jitter_points(base_points, n_times=5, std=0.001, min_total=200):
    jittered = []
    for pt in base_points:
        jittered.append(pt + np.random.normal(scale=std, size=(n_times, 3)))
    jittered = np.vstack(jittered)

    # 최소 수량 미달 시 랜덤하게 추가 생성
    current = jittered.shape[0]
    if current < min_total:
        needed = min_total - current
        selected = base_points[np.random.choice(len(base_points), needed, replace=True)]
        extra_jitter = selected + np.random.normal(scale=std, size=(needed, 3))
        jittered = np.vstack([jittered, extra_jitter])
    return jittered

# 4. jittered 점 생성 (최소 200개)
edge_jittered = jitter_points(edge_points, n_times=5, std=0.101, min_total=500)

# 5. 각각 색상 설정
pcd_original = o3d.geometry.PointCloud()
pcd_original.points = o3d.utility.Vector3dVector(points)
pcd_original.paint_uniform_color([0, 0, 0])  # 검정

pcd_jittered = o3d.geometry.PointCloud()
pcd_jittered.points = o3d.utility.Vector3dVector(edge_jittered)
pcd_jittered.paint_uniform_color([1, 0, 0])  # 빨강

# 6. 시각화
o3d.visualization.draw_geometries([pcd_original, pcd_jittered])

# 7. 저장
o3d.io.write_point_cloud("wing2.xyz", pcd_jittered, write_ascii=True)

True